In [30]:
import requests
import pyotp
import time
import uuid

base_url = "http://localhost:12437"
# base_url = "https://api.test.magicalauth.com"
failures = 0
test_email = f"{uuid.uuid4}ss@devxt.com"


def register_user():
    login_response = requests.post(
        f"{base_url}/v1/user",
        json={
            "email": test_email,
            "first_name": "Test",
            "last_name": "Name",
            "company_name": "Test Company",
            "job_title": "Test Job Title",
        },
    )
    response = login_response.json()
    print(response)
    if "otp_uri" in response:
        mfa_token = response["otp_uri"].split("secret=")[1].split("&")[0]
        totp = pyotp.TOTP(mfa_token)
        return f"Success! Your MFA token is: {totp.now()}"
    else:
        return response


otp_uri = register_user()
print(otp_uri)

{'otp_uri': 'otpauth://totp/MagicalAuth:%3Cfunction%20uuid4%20at%200x0000022295D8DD80%3Ess%40devxt.com?secret=2BYWNMIYC4TMNOA5GMXJRSCZFRGQRV2C&issuer=MagicalAuth'}
Success! Your MFA token is: 953850


In [25]:
import requests

response = requests.get(
    f"{base_url}/v1/user/exists",
    params={"email": "asfasfas@asfafa.com"},
)
print(response.json())

False


In [ ]:
totp_token = otp_uri.split("secret=")[1].split("&")[0]
totp = pyotp.TOTP(totp_token)
print(totp.now())

response = requests.post(
    f"{base_url}/v1/login",
    json={"email": test_email, "token": totp.now()},
)
response = response.json()
if "detail" in response:
    detail = response["detail"]
    if str(detail).startswith("http"):
        token = detail.split("token=")[1]
        print(token)
    else:
        print(response)